In [52]:
import numpy as np
from PIL import Image


class PPG:
    @staticmethod
    def recover(mosaic_image):
        pad_widths = ((2, 2), (2, 2), (0, 0))
        padded_mosaic_image = np.pad(mosaic_image, pad_widths)

        red_layer = padded_mosaic_image[:, :, 0].copy()
        green_layer = padded_mosaic_image[:, :, 1].copy()
        blue_layer = padded_mosaic_image[:, :, 2].copy()
        red_or_blue_layer = red_layer + blue_layer

        # STEP 1 : Get green component for pixels where red or green was defined
        for y in range(2, padded_mosaic_image.shape[0] - 2):
            for x in range(2, padded_mosaic_image.shape[1] - 2):
                if (x + y) % 2 == 0:
                    green_layer[y, x] = PPG._get_green_value(
                        green_layer, red_or_blue_layer, x, y
                    )

        # STEP 2 : Get blue and red components for pixels where both of them were not defined
        for y in range(2, padded_mosaic_image.shape[0] - 2):
            for x in range(2, padded_mosaic_image.shape[1] - 2):
                if (x + y) % 2 != 0:
                    red_layer[y, x], blue_layer[y, x] = PPG._get_red_and_blue_value(
                        green_layer, red_layer, blue_layer, x, y
                    )

        # STEP 3 : Get blue component for pixels where red was defined or opposite
        for y in range(2, padded_mosaic_image.shape[0] - 2):
            for x in range(2, padded_mosaic_image.shape[1] - 2):
                if x % 2 == 0 and y % 2 == 0:
                    blue_layer[y, x] = PPG._get_red_or_blue_value(
                        green_layer, red_or_blue_layer, x, y
                    )
                if x % 2 != 0 and y % 2 != 0:
                    red_layer[y, x] = PPG._get_red_or_blue_value(
                        green_layer, red_or_blue_layer, x, y
                    )

        target_image = np.zeros(
            (
                padded_mosaic_image.shape[0] - 4,
                padded_mosaic_image.shape[1] - 4,
                padded_mosaic_image.shape[2],
            )
        )

        for y in range(2, padded_mosaic_image.shape[0] - 2):
            for x in range(2, padded_mosaic_image.shape[1] - 2):
                target_image[y - 2, x - 2, 0] = red_layer[y, x]
                target_image[y - 2, x - 2, 1] = green_layer[y, x]
                target_image[y - 2, x - 2, 2] = blue_layer[y, x]

        return target_image

    @staticmethod
    def _get_green_value(green_layer, red_or_blue_layer, x, y):
        N = abs(red_or_blue_layer[y, x] - red_or_blue_layer[y - 2, x]) * 2 + abs(
            green_layer[y + 1, x] - green_layer[y - 1, x]
        )

        S = abs(red_or_blue_layer[y, x] - red_or_blue_layer[y + 2, x]) * 2 + abs(
            green_layer[y + 1, x] - green_layer[y - 1, x]
        )

        E = abs(red_or_blue_layer[y, x] - red_or_blue_layer[y, x + 2]) * 2 + abs(
            green_layer[y, x + 1] - green_layer[y, x - 1]
        )

        W = abs(red_or_blue_layer[y, x] - red_or_blue_layer[y, x - 2]) * 2 + abs(
            green_layer[y, x + 1] - green_layer[y, x - 1]
        )

        min_grad = min(N, E, W, S)

        if N == min_grad:
            green = (
                green_layer[y - 1, x] * 3
                + green_layer[y + 1, x]
                + red_or_blue_layer[y, x]
                - red_or_blue_layer[y - 2, x]
            ) / 4
        elif E == min_grad:
            green = (
                green_layer[y, x + 1] * 3
                + green_layer[y, x - 1]
                + red_or_blue_layer[y, x]
                - red_or_blue_layer[y, x + 2]
            ) / 4
        elif W == min_grad:
            green = (
                green_layer[y, x - 1] * 3
                + green_layer[y, x + 1]
                + red_or_blue_layer[y, x]
                - red_or_blue_layer[y, x - 2]
            ) / 4
        elif S == min_grad:
            green = (
                green_layer[y + 1, x] * 3
                + green_layer[y - 1, x]
                + red_or_blue_layer[y, x]
                - red_or_blue_layer[y + 2, x]
            ) / 4

        return PPG._cut_pixel_range(green)

    @staticmethod
    def _hue_transit(L1, L2, L3, V1, V3):
        if L1 < L2 and L2 < L3 or L1 > L2 and L2 > L3:
            return V1 + (V3 - V1) * (L2 - L1) / (L3 - L1)
        return (V1 + V3) / 2 + (L2 - (L1 + L3) / 2) / 2

    @staticmethod
    def _get_red_and_blue_value(green_layer, red_layer, blue_layer, x, y):
        if y % 2 != 0:
            blue = PPG._hue_transit(
                green_layer[y, x - 1],
                green_layer[y, x],
                green_layer[y, x + 1],
                blue_layer[y, x - 1],
                blue_layer[y, x + 1],
            )

            red = PPG._hue_transit(
                green_layer[y - 1, x],
                green_layer[y, x],
                green_layer[y + 1, x],
                red_layer[y - 1, x],
                red_layer[y + 1, x],
            )

        else:
            blue = PPG._hue_transit(
                green_layer[y - 1, x],
                green_layer[y, x],
                green_layer[y + 1, x],
                blue_layer[y - 1, x],
                blue_layer[y + 1, x],
            )

            red = PPG._hue_transit(
                green_layer[y, x - 1],
                green_layer[y, x],
                green_layer[y, x + 1],
                red_layer[y, x - 1],
                red_layer[y, x + 1],
            )

        return PPG._cut_pixel_range(red), PPG._cut_pixel_range(blue)

    @staticmethod
    def _get_red_or_blue_value(green_layer, red_or_blue_layer, x, y):
        NE = (
            abs(red_or_blue_layer[y - 1, x + 1] - red_or_blue_layer[y + 1, x - 1])
            + abs(red_or_blue_layer[y - 2, x + 2] - red_or_blue_layer[y, x])
            + abs(red_or_blue_layer[y, x] - red_or_blue_layer[y + 2, x - 2])
            + abs(green_layer[y - 1, x + 1] - green_layer[y, x])
            + abs(green_layer[y, x] - green_layer[y + 1, x - 1])
        )

        NW = (
            abs(red_or_blue_layer[y - 1, x - 1] - red_or_blue_layer[y + 1, x + 1])
            + abs(red_or_blue_layer[y - 2, x - 2] - red_or_blue_layer[y, x])
            + abs(red_or_blue_layer[y, x] - red_or_blue_layer[y + 2, x + 2])
            + abs(green_layer[y - 1, x - 1] - green_layer[y, x])
            + abs(green_layer[y, x] - green_layer[y + 1, x + 1])
        )

        if NE < NW:
            red_or_blue = PPG._hue_transit(
                green_layer[y - 1, x + 1],
                green_layer[y, x],
                green_layer[y + 1, x - 1],
                red_or_blue_layer[y - 1, x + 1],
                red_or_blue_layer[y + 1, x - 1],
            )

            return PPG._cut_pixel_range(red_or_blue)

        red_or_blue = PPG._hue_transit(
            green_layer[y - 1, x - 1],
            green_layer[y, x],
            green_layer[y + 1, x + 1],
            red_or_blue_layer[y - 1, x - 1],
            red_or_blue_layer[y + 1, x + 1],
        )

        return PPG._cut_pixel_range(red_or_blue)

    @staticmethod
    def _cut_pixel_range(pixel):
        pixel = 0 if pixel < 0 else pixel
        pixel = 255 if pixel > 255 else pixel
        return pixel


def PSNR(original, recovered):
    cfa_original = (
        0.299 * original[:, :, 0]
        + 0.587 * original[:, :, 1]
        + 0.114 * original[:, :, 2]
    )
    cfa_recovered = (
        0.299 * recovered[:, :, 0]
        + 0.587 * recovered[:, :, 1]
        + 0.114 * recovered[:, :, 2]
    )
    mse = np.mean((cfa_original - cfa_recovered) ** 2)
    return 10 * np.log10(255.0**2 / mse)

In [54]:
original = Image.open("Original.bmp")
mosaic = Image.open("RGB_CFA.bmp")

original = np.asarray(original, dtype=np.int64)
mosaic = np.asarray(mosaic, dtype=np.int64)

In [55]:
recovered = PPG.recover(mosaic)

In [59]:
print(f"PSNR = {PSNR(original, recovered)}")
print(f"Image size = {original.size / 10**6} MP")

PSNR = 25.176288669592545
Image size = 25.970544 MB


### Result

PSNR : 25.7

Original image resolution : ~16

Recovered image resolution : ~8

Algorithm work time : 2m 22.4s = 142.4s

Image size : 25.970544 MP

Speed : 142.4s / 25.970544 MP = 5,468 s/MP